需要資料 :
- 上一個月的營收 (營收資料 202401-202403.csv)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import json
import time

In [ ]:
# token = 'UV04qrPBUJpWCWMseXJA4qkBOnoFzJRaBRQsnGQ95NQ'  # 群組:test # 營收通知
# token2 = 'daRhrufmVS355l1yFVyLeJdmRqNjohWGN17cv0n33Mi'  # 群組:營收通知 # 營收通知

In [2]:
# fm3k26
df_revenue = pd.read_csv('營收資料 202401-202403.csv', encoding='utf-16', sep='\t')
df_revenue.insert(0, column='scode', value=df_revenue['證券代碼'].apply(lambda x : x.split(' ')[0]))
df_revenue.insert(1, column='sname', value=df_revenue['證券代碼'].apply(lambda x : x.split(' ')[1]))
df_revenue.drop(columns=['證券代碼'], inplace=True)
df_revenue = df_revenue.sort_values(['scode', '年月']).reset_index(drop=True)

df_revenue['last_month_revenue'] = df_revenue['單月營收(千元)'].shift(1)
df_revenue.loc[df_revenue['年月']=='2024/01', 'last_month_revenue'] = np.nan

df_revenue['MoM'] = round((df_revenue['單月營收(千元)']-df_revenue['last_month_revenue'])/df_revenue['last_month_revenue']*100, 2)

# 調整
df_revenue.loc[df_revenue['單月營收(千元)']>df_revenue['歷史最高單月營收(千元)'], '歷史最高單月營收(千元)'] = df_revenue.loc[df_revenue['單月營收(千元)']>df_revenue['歷史最高單月營收(千元)'], '單月營收(千元)']
df_revenue.loc[df_revenue['單月營收(千元)']<df_revenue['歷史最低單月營收(千元)'], '歷史最低單月營收(千元)'] = df_revenue.loc[df_revenue['單月營收(千元)']<df_revenue['歷史最低單月營收(千元)'], '單月營收(千元)']
df_revenue

,scode,sname,年月,單月營收(千元),歷史最高單月營收(千元),歷史最低單月營收(千元),last_month_revenue,MoM
0,000930,華南永昌證券,2024/01,508334.0,849956,-4478047,NaN,NaN
1,000930,華南永昌證券,2024/02,466635.0,849956,-4478047,508334.0,-8.20
2,000930,華南永昌證券,2024/03,703898.0,849956,-4478047,466635.0,50.85
3,000960,富邦證券,2024/01,1831101.0,2250900,-468561,NaN,NaN
4,000960,富邦證券,2024/02,1602660.0,2250900,-468561,1831101.0,-12.48
...,...,...,...,...,...,...,...,...
6691,9960,邁達康,2024/02,47141.0,122107,24259,49458.0,-4.68
6692,9960,邁達康,2024/03,49229.0,122107,24259,47141.0,4.43
6693,9962,有益,2024/01,207195.0,441721,98154,NaN,NaN
6694,9962,有益,2024/02,149766.0,441721,98154,207195.0,-27.72


In [ ]:
# line notify
def lineNotifyMessage(token, msg):
    headers = {
        "Authorization": 'Bearer ' + token, 
        # "Content-Type" : "application/x-www-form-urlencoded"
    }

    data = {'message': msg}
    
    try:
        # r = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data, files=files)
        r = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)
        r.raise_for_status()  # 如果發生錯誤，會丟出相應的異常
        return r.status_code
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None
    
token = 'UV04qrPBUJpWCWMseXJA4qkBOnoFzJRaBRQsnGQ95NQ'

# ==================================================
def mops_revenue_notify(df_revenue, token, last_begin_time):
    url = "https://mops.twse.com.tw/mops/web/t51sb10"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    response = requests.get(url+'?Stp=R1&TK=', headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    
    for i in range(100):
        try:
            action = soup.find_all("tr", class_=["even","odd"])[i].button['onclick'].split('\'')[1]
            title = soup.find_all("tr", class_=["even","odd"])[i].find_all('u')[0].text
            time = title.split('(')[0]
        except:
            print(i, 'wrong')
            break
        
        if i == 0:
            begin_time = time
        
        # 判斷是否重複讀取過
        if time <= last_begin_time:
            print(i)
            break
        # print(title)
        # 如果是營業收入 & 上市上櫃 才做
        if 'ifrs' in action and ('上市' in title or '上櫃' in title):
            print(title)
            try:
                lst_find_input = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')
                TYPEK = lst_find_input[0].get('value')
                firstin = lst_find_input[1].get('value')
                co_id = lst_find_input[2].get('value')
                off = lst_find_input[3].get('value')
                year = lst_find_input[4].get('value')
                month = lst_find_input[5].get('value')
                step = lst_find_input[6].get('value')

                payload = {
                    "TYPEK": TYPEK,
                    "firstin": firstin,
                    "co_id": co_id,
                    "off": off,
                    "year": year,
                    "month": month,
                    "step": step
                }
                # print(payload, action)

                res = requests.post(url+action, data=payload).content
                content_soup = BeautifulSoup(res, "html.parser")
                # print(content_soup.prettify())


                # -----------------------------------
                sname = title.split(')')[1].split('(')[0]
                year = str(int(year)+1911)

                # ----------------------
                # 取出營收
                now_revenue = float(content_soup.find_all("td", class_=["even","odd"])[0].text.split(' ')[-1].replace(',', ''))

                # ----------------------
                # YoY
                # last_year_revenue = float(content_soup.find_all("td", class_=["even","odd"])[1].text.split(' ')[-1].replace(',', ''))
                # YoY = round((now_revenue-last_year_revenue)/last_year_revenue*100, 2)
                if 'KY' in title:
                    YoY = float(content_soup.find_all("td", class_=["even","odd"])[6].text.split(' ')[-1].replace(',', ''))
                else:
                    YoY = float(content_soup.find_all("td", class_=["even","odd"])[3].text.split(' ')[-1].replace(',', ''))

                # ----------------------
                # MoM   # 歷史新高/新低
                last_month = str(int(month)-1)
                if len(last_month)==1:
                    last_month = '0'+last_month

                remark = ''
                cond_code = df_revenue['scode']==co_id
                cond_yymm = df_revenue['年月']==(year+'/'+last_month)
                if not df_revenue[cond_code&cond_yymm].empty:
                    MoM = round((now_revenue-df_revenue[cond_code&cond_yymm]['單月營收(千元)'].iloc[0])/df_revenue[cond_code&cond_yymm]['單月營收(千元)'].iloc[0]*100, 2)
                    hist_high_revenue = float(df_revenue[cond_code&cond_yymm]['歷史最高單月營收(千元)'].iloc[0])
                    hist_low_revenue = float(df_revenue[cond_code&cond_yymm]['歷史最低單月營收(千元)'].iloc[0])
                    if now_revenue > hist_high_revenue:
                        remark = '\n\n創歷史新高!!!'
                    elif now_revenue < hist_low_revenue:
                        remark = '\n\n創歷史新低!!!'
                    else:
                        remark = ''
                else:
                    MoM = '沒有 MoM'

                # -------------------------
                # 備註(異常原因)
                if 'KY' in title:
                    try:
                        reason = content_soup.find_all("td", class_=["even","odd"])[20].text.replace('\xa0', '')
                    except:
                        reason = content_soup.find_all("td", class_=["even","odd"])[8].text
                else:
                    reason = content_soup.find_all("td", class_=["even","odd"])[8].text

                if reason == '' or reason == '無':
                    reason = ''
                else:
                    reason = '\n\n異常原因: ' + reason

    #             print('============================')
                print(co_id, sname, title.split('(')[0])
    #             print(year+'/'+month, '營收(億)', now_revenue)
    #             print('YoY', YoY, '%')
    #             print('MoM', MoM, '%')
    #             print(remark)
    #             print('異常原因:', reason)

                msg = ('\n'+
                       co_id+' '+sname+'  '+title.split('(')[0] + '\n' +
                       year+'/'+month+' 營收(億) '+str(now_revenue/100000) + '\n' +
                       'YoY '+str(YoY)+' %' +'  '+'MoM '+str(MoM)+' %'
                       +reason
                       +remark
                      )
                lineNotifyMessage(token, msg)
            
            
                # 另一個群組
                token2 = 'daRhrufmVS355l1yFVyLeJdmRqNjohWGN17cv0n33Mi'  # 群組:營收通知 # 營收通知
                lineNotifyMessage(token2, msg)
            except:
                print(co_id, sname, title.split('(')[0], 'wrong')
    return begin_time

# ==================================================
# main
for i in range(600):
    if i == 0:
        last_time = '17:49'
    print('============================')
    print(datetime.now().strftime("%H:%M"))
    print(last_time, 'excute')    
    temp = mops_revenue_notify(df_revenue, token, last_time)
    print(last_time+'~'+temp, 'done')
    
    last_time = temp
    
    if datetime.now().strftime("%H:%M") < '06:00':
        break
    
    if datetime.now().strftime("%H:%M") > '14:30':
        time.sleep(300)
    else:
        time.sleep(60)
        
# 結束執行 通知
token = 'UV04qrPBUJpWCWMseXJA4qkBOnoFzJRaBRQsnGQ95NQ'  # 群組:test # 營收通知
msg = '營收通知 結束執行'
lineNotifyMessage(token, msg)

18:00
17:49 excute
17:58(上櫃公司)寶得利(5301)-11304-營業收入資訊
5301 寶得利 17:58
17:57(上櫃公司)基亞(3176)-11304-營業收入資訊
3176 基亞 17:57
14
17:49~18:00 done
18:05
18:00 excute
18:05(上櫃公司)桂田文創(4806)-11304-營業收入資訊
4806 桂田文創 18:05
18:03(上櫃公司)辣椒(4946)-11304-營業收入資訊
4946 辣椒 18:03
18:02(上櫃公司)寶碩(5210)-11304-營業收入資訊
5210 寶碩 18:02
11
18:00~18:05 done
18:10
18:05 excute
18:10(上櫃公司)奇鈦科(3430)-11304-營業收入資訊
3430 奇鈦科 18:10
18:08(上市公司)元大金(2885)-11304-營業收入資訊
2885 元大金 18:08 wrong
7
18:05~18:10 done
18:16
18:10 excute
18:11(上市公司)裕隆(2201)-11304-營業收入資訊
2201 裕隆 18:11
11
18:10~18:13 done
18:21
18:13 excute
18:20(上櫃公司)霹靂(8450)-11304-營業收入資訊
8450 霹靂 18:20
18:16(上櫃公司)智基(6294)-11304-營業收入資訊
6294 智基 18:16
12
18:13~18:20 done
18:26
18:20 excute
18:22(上市公司)大亞(1609)-11304-營業收入資訊
1609 大亞 18:22
7
18:20~18:26 done
18:31
18:26 excute
1
18:26~18:27 done
18:36
18:27 excute
1
18:27~18:35 done
18:41
18:35 excute
18:40(上櫃公司)富裔(6264)-11304-營業收入資訊
6264 富裔 18:40
6
18:35~18:40 done
18:46
18:40 excute
18:44(上櫃公司)達邦蛋白(6578)-11304-營業收入資訊
6578 達邦蛋白 18:44
6
18

## 研究

In [8]:
url = "https://mops.twse.com.tw/mops/web/t51sb10"
response = requests.get(url+'?Stp=R1&TK=')
soup = BeautifulSoup(response.text, "html.parser")

In [9]:
i = 2
soup.find_all("tr", class_=["even","odd"])[i].find_all('u')[0]

<u>08:29(上市公司)榮成(1909)-11304-營業收入資訊</u>

In [83]:
i = 4

title = soup.find_all("tr", class_=["even","odd"])[i].find_all('u')[0].text
print(title)
TYPEK = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')[0].get('value')
firstin = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')[1].get('value')
co_id = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')[2].get('value')
off = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')[3].get('value')
year = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')[4].get('value')
month = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')[5].get('value')
step = soup.find_all("tr", class_=["even","odd"])[i].find_all('input')[6].get('value')

action = soup.find_all("tr", class_=["even","odd"])[i].button['onclick'].split('\'')[1]
if 'ifrs' in action :
    print(payload, action)

payload = {
    "TYPEK": TYPEK,
    "firstin": firstin,
    "co_id": co_id,
    "off": off,
    "year": year,
    "month": month,
    "step": step
}

res = requests.post(url+action, data=payload).content
content_soup = BeautifulSoup(res, "html.parser")
print(content_soup.prettify())

20:16(上市公司)宅配通(2642)-11304-營業收入資訊
{'TYPEK': 'otc', 'firstin': 'Y', 'co_id': '6576', 'off': '1', 'year': '113', 'month': '01', 'step': '1'} /mops/web/ajax_t05st10_ifrs
<html>
 <head>
  <title>
   公開資訊觀測站
  </title>
  <link href="css/css2.css" media="Screen" rel="stylesheet" type="text/css"/>
  <script src="js/mops2.js" type="text/javascript">
  </script>
 </head>
 <body>
  <tr>
   <td align="left" class="reportCont">
    <h3>
     <u>
      <a alt="查詢彙總報表" href="/mops/web/t21sc04_ifrs">
       查詢彙總報表
      </a>
     </u>
    </h3>
   </td>
  </tr>
  <table class="noBorder" style="width: 60%; margin-left: 20%;">
   <tr>
    <td align="right">
     <form action="/server-java/t56ques" method="post">
      <input name="step" type="hidden" value="0"/>
      <input name="Market" type="hidden" value="sii "/>
      <input name="SysName" type="hidden" value="營運概況"/>
      <input name="reportName" type="hidden" value="開立發票及營業收入資訊"/>
      <input name="colorchg" type="hidden" value=""/>
      <inp

In [120]:
now_revenue = float(content_soup.find_all("td", class_=["even","odd"])[0].text.split(' ')[-1].replace(',', ''))

last_year_revenue = float(content_soup.find_all("td", class_=["even","odd"])[1].text.split(' ')[-1].replace(',', ''))

YoY = round((now_revenue-last_year_revenue)/last_year_revenue*100, 2)
# YoY = float(content_soup.find_all("td", class_=["even","odd"])[3].text.split(' ')[-1].replace(',', ''))

2.3

In [106]:
# content_soup.find_all("th", class_='tblHead')[2:]    # 標題
# content_soup.find_all("td", class_=["even","odd"])   # 營收數值

print(title)
for i in range(len(content_soup.find_all("th", class_='tblHead')[2:])):
    print(content_soup.find_all("th", class_='tblHead')[2:][i].text, content_soup.find_all("td", class_=["even","odd"])[i].text)

20:16(上市公司)宅配通(2642)-11304-營業收入資訊
本月                324,673
去年同期                317,366
增減金額                  7,307
增減百分比         2.30
本年累計               1,346,140
去年累計              1,325,218
增減金額                  20,922
增減百分比         1.58
備註 / 營收變化原因說明 
